# Faze 1 And 2 

## Amirhossein Rezazade / Elham Ghorbani

### Libraries

در این بخش ابتدا تمام کتابخانه های مورد نیاز خود را اضافه میکنیم 

In [2]:
import numpy as np, hazm, pandas as pd,sys , collections,math
from hazm import *
import codecs
import xml.etree.ElementTree as ET
from bplustree import BPlusTree

In [3]:
#path ="C:/Users/Asus/Desktop/nlp/PR/"
path =""

### A model for reading data
کلاس آیتم یک کلاس مدل برای ذخیره اطلاعات لازم و پارس کردن دیتای اصلی میباشد

In [4]:
class Item:
   def __init__(self):
    self.text = ''
    self.date = ''
    self.cat = ''
    self.titel = ''
    self.id = 0

### Reading XML File and Creating Arry of items
در این بخش ابتدا لیستی خالی برای پر کردن دیتا درست میکنیم و قایل دیتای اصلی را باز میکنیم

In [5]:
MyData = []
tree = ET.parse(path+'data.xml')
root = tree.getroot()

این بخش با استفاده از تگ های موجود در فایل توشته شده است و به ان صورت توانستیم دیتای اصلی را پارس کرده و در لیست قرار دهیم 
در همین زمان تمام آیدی ها را خودمان به عدد تبدیل میکنیم تا بتوانیم با آیدی های عددی کار کنیم

In [6]:

id = 0
for child in root: # DOC
    myItem = Item()
    for child1 in child:  
        if child1.tag == "TEXT":
            myItem.text = child1.text
        if child1.tag == "ISSUE":
            myItem.date = child1.text
        if child1.tag == "TITLE":
            myItem.titel = child1.text
        if child1.tag == "CAT":
            myItem.cat = child1.text
        if child1.tag == "DOCID":
            #myItem.id = child1.text
            myItem.id = id
            id+=1
    MyData.append(myItem)

در اینجا برای مقال زمان مربوط به سند هفتم را چاپ میکنیم

In [7]:
print(MyData[7].date)

دوشنبه 11 دي 1385 - سال چهاردهم - شماره4172 - Jan 1, 2007


### Read Removable charecters and stop words from file 
دراین بخش فایلی  به نام کاراکتر های غیر مجاز داریم که باید این کاراکتر ها را حذف کنیم و از آن جایی که هر کاراکتر در خطی ذخیره شده است پس اندیس صفر ام هر خط را خوانده و در لیستی قرار میدهیم

In [8]:
rlist = codecs.open(path+"Removable charecters.txt", encoding='utf-8').read().split('\n')
# rlist is arry of Removable charecters
removeable = [c[0] for c in rlist]

برای مثال کاراکتر 18 ام را چاپ میکنیم

In [9]:
print(removeable[18])

꧁


در اینجا همان عمل را برای کلمات توقف انجام میدهیم و تمام آن ها را بار گیری میکنیم و در نهایت لیستی از آن درست میکنیم

In [10]:
nmz = Normalizer()
wordTokenizer = WordTokenizer()
lemmatizer = Lemmatizer()
slist = codecs.open(path+'Stops.txt', encoding='utf-8').read().split('\n')
stops = sorted(list([nmz.normalize(w) for w in slist if w]))

برای مقال کلمه توقف 25 ام را نشان میدهیم 

In [11]:
print(stops[25])

آره


### Using Normalizer,WordTokenizer and Lemmatizer Class
#### we can see usage from this url : https://github.com/sobhe/hazm

در تابع زیر ما یک متن به ورودی میدهیم و از آن لیستی از کلمات را مییگریم که این کلمات نرمال شده و تمام کلمات توقف از آن حذف گردیده است

In [12]:
def Process(TEXT):        
    TEXT = ''.join([c for c in TEXT if c not in removeable])
    TEXT = nmz.character_refinement(TEXT) # pack kardan kalamat arabi va adad englisi
    TEXT = nmz.affix_spacing(TEXT) # tabdile fasele be nim fasele                 
    tokens = wordTokenizer.tokenize(TEXT)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stops]                     
    tokens = set(tokens)
    tokens = sorted(list(tokens))        
    return tokens

برای مقال عبارت زیر را به تابع میدهیم

In [13]:
print(Process('سهام یا بورس و پول'))

['بورس', 'سهام', 'پول']


در اینجا تمام استاد را پیمایش میکنیم و متن تک تک آن ها را با استفاده از تابع خود به لیتستی از توکن ها تبدیل میکنیم و در مدل جدید خود قرار میدهیم

In [14]:
class TokenItem:
  def __init__(self):
    self.DocId = 0
    self.Tokens = []

TokenList = []

for item in MyData:
    t = TokenItem()
    t.DocId = item.id
    t.Tokens = Process(item.text)
    TokenList.append(t)

در نهابت لیستی که داریم شامل تمام آیدی ها به همراه توکن های آن هاست

### example
چاپ آرایه 15 از توکن لیستمان 


In [15]:
print (TokenList[15].Tokens)

['آذرماه', 'آلات', 'آورد#آور', 'ارزش', 'از\u200e', 'اساس', 'استدر', 'افت', 'افزایش', 'افزود#افزا', 'الف', 'املاک', 'انبوه', 'اوراق', 'ایجاد', 'ایران', 'ب', 'بازار', 'بازارساز', 'بازیگر', 'بانک', 'بزرگ', 'بلندمدت', 'بهادار', 'بورس', 'بورسهای', 'تاسیس', 'تجهیزات', 'تدریجی', 'تصحیح', 'تصمیم', 'تعادل', 'تعدیل', 'توجه', 'توس', 'تولید', 'جمع', 'جمله', 'حفظ', 'حمایت', 'خاص', 'خروج', 'خرید', 'خریدار', 'خصوصی', 'خیز', 'دادطی', 'داشت#دار', 'درصد', 'دنیا', 'روند', 'زنده', 'زیرمجموعه', 'سال', 'سدید', 'سرمایه', 'سزا', 'سموم', 'سهام', 'سهامدار', 'سهم', 'سود', 'سودآوری', 'سیستم', 'شرکت', 'شیما', 'شیمیایی', 'علف', 'عملکرد', 'غیررسمی', 'فروش', 'فعالیت', 'فلزات', 'قبل', 'لوله', 'ماشین', 'مالکیت', 'مالی', 'متعادل', 'مجموعه', 'محدود', 'محدودی', 'محدودیت', 'محصولات', 'مرور', 'مستغلات', 'منابع', 'موثر', 'موثری', 'نظام', 'نقش', 'نگه', 'وضعیت', 'پارس', 'پاسارگاد', 'پررنگ', 'پول', 'چشمگیر', 'کارگیری', 'کاهش', 'کرد#کن', 'کسب', 'کش', 'کشور', 'کمباین', 'گذاری\u200cهای', 'گروه', 'گزارش', 'گستر']


### posting
 
اول امدیم پستینگ را به عنوان دیکشنری  تعریف کردیم ،و تابعی به نام ادداک که دو آرگومان ،توکن و داک آیدی داره را نوشته و پستینگ را به عنوان متغییر سراسری گرفتیم و گفتیم که اگر توکن در پستینگ ما نبود داک آیدی را اضافه کن به پستینگ ما و تابعی به نام اساینمنت  با آرگمان توکن ایتم داریم و با استفاده از کتابخونه پانداس  آمدیم سریس که مشابه ارایه یک بعدی که داده ها را از هر نوعی ذخیره می کند که مقادیر  قایل تغییر اما اندازه غیر قابل تغییر داردبرای  توکن آیتم داک ایدی و توکن  بدست آورده و در آخر اجرا کردیم با لمبداکه همان حلقه فور اما سریع تر ،در نهایت با حلقه دونه دونه کلمات را بگیره و به ما پستینگش را بده .            

In [16]:
Posting = {}
def addDoc(token,docID):
    global Posting
    if(token not in Posting):
        Posting[token] = []
    Posting[token].append(docID)                               
    
def Assignment(tokenitem): 
    pd.Series(tokenitem.Tokens).apply(lambda token: addDoc(token, tokenitem.DocId));        


for i in TokenList:
    Assignment(i)

for v in Posting.values():
    v = sorted(v)

### example
چاپ  پستینگ کلمه مورد نظر  


In [17]:
print(Posting["ادب"])

[5, 7, 11, 85, 95]


### query 


در این لخش به یک تابع برای پرس و جو تعریف میکنیم که همان تابع اصلی برنامه ماست که سرویس جسجو را به برنمه کلاینت میدهد در این تابع ابتدا یک متن گرفته شده و با همان تابع نرمال کننده به لبستی از توکن ها تبدیل میشود و هر توکن جداگانه پرس وجو میشود
در نهایت همه پرس و جو ها با هم اشتراک گرفته میشوند

In [18]:
def Query(Text):
    try:
        WordTokens = Process(Text)
        posting_List = []
        for word in WordTokens:
            posting = Posting[word]
            posting_List.append(posting)

        if(len(posting_List) == 0):
            return []

        FInalPosting = posting_List[0]

        for plist in posting_List:
            FInalPosting = set(FInalPosting).intersection(plist)
            
        return sorted(FInalPosting)        
    except:
        return []

### example
پستیمگ یک ترم

In [19]:
print(Posting["پول"])

[15, 21, 54, 86]


برای مثال تمام پستینگ های یک عبارت این گونه است

In [20]:
myText = "سهام یا بورس و پول"
tk = Process(myText)
for t in tk:
    print(t + " : " + str(Posting[t]))

بورس : [13, 15, 16, 17, 18]
سهام : [13, 14, 15, 16, 17, 18, 53, 91]
پول : [15, 21, 54, 86]


تابع پرس و جوی ما باید اشتراک این سه مجموعه را برگرداند

In [21]:
print(Query('سهام یا بورس و پول'))

[15]


در انتها سه تایع زیر برای استفاده در رابط کاربری نرم افزار استفاده میگردد
 اولی برای گرفتن لیست مناسبی از نتیجه پرس و چو و دومی برای گرفتن اطلاعات یک سند به صورت انفرادی
سومس هم برای دادن لیست پیشنهادی به کاربر از طریق پستینگ ها

تابع پرس چو برای کلاینت

In [22]:
def GetDocList(Text):
    posintg = Query(Text)
    DocList = []
    for docId in posintg:
        for item in MyData:
            if(docId == item.id):
                jData = {
                    'Id' : item.id,
                    'Category' : item.cat,
                    'Titel' : item.titel,
                    'Text' : item.text[0:20] + " ...",
                    'Date' : item.date
                }
                DocList.append(jData)
    return DocList

تابع گرفتن اطلاعات یک سند برای کلاینت

In [23]:
def GetDoc(DocId):
    for item in MyData:
        if(item.id == DocId):
            return {
                'Id' : item.id,
                'Category' : item.cat,
                'Titel' : item.titel,
                'Text' : item.text,
                'Date' : item.date
            }
    return {
            'Id' : DocId,
            'Category' : '',
            'Titel' : '',
            'Text' : '',
            'Date' : ''
        }


تابع دادن پیشنهاد به کاربر از طریق پستینگ

In [24]:
def GetSuggestion(Text):
    w = Text.split(' ')
    last = w[len(w)-1]
    index = Text.index(last)
    res = []
    for i in Posting.keys():
        if(i.startswith(last)):
            res.append(Text[0:index] + i)
    return res

### در هم سازی

تایع هش ما یک کلمه میگیرد و یک عدد بین صفر تا عدد مورد نظر را به ما میدهد 

In [25]:
def h(str,max):
    num = hash(str)
    return num % max

برای مثال

In [26]:
print(h("hi",100))

70


حالا تعداد تمام ترم های موجود در دیکشنری را میبینیم 

In [27]:
Count = len(Posting.keys())
print(Count)

4809


آرایه ای به طول تمام کلمات خود ایجاد میکنیم تا کلمات خود را در آرایه بریزیم

In [28]:

arr = []
for i in range(Count):
    arr.append([])


for k in Posting.keys():
    index = h(k,Count)
    arr[index].append(k)


تایع جستجو در این آرابه در هم سازی شده به صورت زیر است

In [29]:
def Find(Word):
    find_index = h(Word,Count)
    return (Word in arr[find_index])

جستحو بسیار سریع

In [30]:
my_word = "یکدست"

print(Find(my_word))

True


تست و ارزیابی

میدانیم که بر ای همه کلمات موچود در دیکشنری جا داریم پس در حالت ایده آل نباید جای خالی در آرایه وجود داشته باشد

In [31]:
CountLIst = []
for x in arr:
    CountLIst.append(len(x))

Temp = sorted(set(CountLIst))

for x in Temp:
    print(
     str(x) +
      "  |   " +
       str(CountLIst.count(x)) +
        "   : " +
         str(round(CountLIst.count(x) / Count * 100,2)) + " %")


0  |   1798   : 37.39 %
1  |   1727   : 35.91 %
2  |   882   : 18.34 %
3  |   311   : 6.47 %
4  |   72   : 1.5 %
5  |   17   : 0.35 %
6  |   2   : 0.04 %


مثال

In [32]:
my_word = "سلام"

find_index = h(my_word,Count)

print(arr[find_index])

[]


درخت

https://www.programmersought.com/article/77574639100/

In [33]:
import B_Tree

برای پرس جوی کلمات * دار، باید تابعی داشته باشیم که عبارت بعد از کلمه را بدهد

مثلا عبارت سهان بعد از کلمه سهام است

In [34]:
def GetNextWord(Word):
    last = Word[-1]
    x = chr(ord(last) + 1)
    return Word[0:len(Word)-1] + x

print(GetNextWord("سهام"))


سهان


برای این که مقدار و پستیگ های خود را در درخت قرار دهیم کلاس زیر را ایجاد میکنیم

In [35]:
class PData(object):
    def __init__(self, v, p):
        self.Term = v
        self.Term_Posting = p
    
    def __repr__(self):
        return self.Term

    def __str__(self):
        return self.Term


    def __eq__(self, other):
        if type(other) != PData:
            return other == self.Term
        return other.Term == self.Term

    def __lt__(self, other):
        if type(other) != PData:
            return other > self.Term
        return other.Term > self.Term

    def __le__(self, other):
        if type(other) != PData:
            return other <= self.Term
        return other.Term <= self.Term

    def __gt__(self, other):
        if type(other) != PData:
            return other < self.Term
        return self.Term > other.Term

    def __ge__(self, other):
        if type(other) != PData:
            return other <= self.Term
        return self.Term >= other.Term

    def __ne__(self, other):
        if type(other) != PData:
            return other != self.Term
        return self.Term != other.Term

تست کلاس

In [36]:
v1 = PData("a",[])
v2 = PData("a",[])

print(v1)
print (v1>v2)
print(v1<v2)
print(v1==v2)
print(v1=="a")

a
False
False
True
True


حالا باید تمام پستیگ خود را به درخت تبدیل کنیم

In [37]:
scala = 2


forward = B_Tree.BTree(scala)               
backward = B_Tree.BTree(scala)

for term in Posting.keys():
    posting = Posting[term]
    forward.insert (PData(term      , posting))       
    backward.insert(PData(term[::-1], posting))                  

پیدا کردن کلمه ای مشخص

In [38]:
s = "سهام"
res = forward.search(s)

print(res.Term + " "  + str(res.Term_Posting))

سهام [13, 14, 15, 16, 17, 18, 53, 91]


جستجو کلمه پیشوندی

In [39]:
s = "ب"
Word_Next = GetNextWord(s)
res = forward.GetValue(s,Word_Next)

for i in res:
    print(i)

باب
بابا
بابک
باتجربه‌ترین‌های
باجه
باخت#باز
باخترینه
باران
بارز
بارسلون
بارسلونا
بارقه
بارور
باریک
باریکه
بازارارزش
بازبینی
بازجویی
بازده
بازدید
بازرسی
بازشدن
بازماندگان
بازنده
بازنشسته
بازنشستگی
بازو
بازپرداخت
بازگردیم
بازگشتن
بازگشتی
بازگفتی
بازگو
بازیکن
بازیکنانش
باستان
باستانی
باسک
باشداز
باشددر
باشکوه
باشیمبعد
باطله
باغ
باغبان
باقرعاظهار
بال
بالاستتعرفه
بالقوه
بالینیزنان
بامانیز
بامدادان
بامعنا
بانو
باهنر
باورناپذیر
باکس
باید
ببر
ببرد
ببری
ببینی
بتوزند
بتون
بحران
بختیار
بخشش
بخشید#بخش
بد
بدان
بدخواه
بدل
بدلیل
بدهکار
بدکنش
بدیل
بدیهی
برآنیم
برآورده
برابرآثار
برادر
برافروزند
برانگیخت
برایتان
بربافته‌اند
برتر
برترین‌های
برجسته
برخواهند
برخورد
برد
برد#بر
بردار
برداشت
بردن
بردگی
بردیاب
بررسد
برزان
برساخته‌اند
برسانددر
برشمارم
برطرف
برف
برقرار
برقراری
برقی‌ها
برمی
برنج
برهانی
برودمنچستر
برون
برپایه
برپایی
برکشیدن
برکنار
برگرداندن
برگردانده
برگردم
برگزارخواهند
برگزیدند
برگزیده
بریتانیا
بزرگ
بزرگراه
بست#بند
بسترسازی
بسته
بستگی
بسر
بسنجد
بسکتبال
بشرطها
بشریت
بعث
بعدشنبههم
بغداد
بغدادخبر
ب

جستجو کلمه پسوندی

In [40]:
s = "بار"
s = s[::-1]
w = GetNextWord(s)
res = backward.GetValue(s,w)

for i in res:
    print(i.Term[::-1])

اعتبار
اخبار
دربار
تأسفبار
اسفبار
انبار


جستجو میانوندی و اشتراک گیری میان دو جواب درخت ها

In [41]:
s = "س"
e = "ار"

w = GetNextWord(s)
fres = forward.GetValue(s,w)
e = e[::-1]
w = GetNextWord(e)
bres = backward.GetValue(e,w)

for i in fres:
    for j in bres:
        if i.Term == j.Term[::-1]:
            print(i)


ساختار
سازگار
سیگار


لیست کلمات فارسی

در قایل زیر مجموعه ای از کلمات فارسی قرار گرفته است

In [90]:
Word_as_Text = codecs.open(path+'Words.txt', encoding='utf-8').read()
    

In [112]:
def GetWords(TEXT):        
    TEXT = ''.join([c for c in TEXT if c not in removeable])
    TEXT = nmz.character_refinement(TEXT) # pack kardan kalamat arabi va adad englisi                
    tokens = wordTokenizer.tokenize(TEXT)                  
    tokens = set(tokens)
    tokens = sorted(list(tokens))        
    return tokens

با تابع بالا کلمات درون فایل متنی را استخراج میکنیم

In [113]:
Words = GetWords(Word_as_Text) + stops

تعداد همه کلمات

In [114]:
print(len(Words))

31458


### Levenshtein


حداقل فاصله ویرایش 


برکرفته از https://python-course.eu/applications-python/levenshtein-distance.php

حداقل فاصله ویرایش بین دو رشته، حداقل تعداد عملیات ویرایش مورد نیاز برای تبدیل یک رشته به رشته دیگر است. عملیات ویرایش می تواند شامل درج، حذف و جایگزینی باشد.

تابع پایتون زیر فاصله لوناشتاین را پیاده سازی می کند:

In [115]:
def iterative_levenshtein(s, t, print_matrix = False):
    costs=(1, 1, 1)
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    if(print_matrix):
        for r in range(rows):
            print(dist[r])
    
 
    return dist[row][col]

مثال

In [116]:
print(iterative_levenshtein("abc", "xyz",True))

[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
3


تابع زیر یک کلمه گرفته و نزدیک ترین پیشنهاد را برمیگرداند

In [122]:
def GetTrueWord(token):
    min = 999999
    min_word = ""
    for item in Words:
        d = iterative_levenshtein(token, item)
        if(min > d):
            min = d
            min_word = item
    return min_word

مثال

In [124]:
print(GetTrueWord("سزو"))

جزو


تابع زیر یک عبارت را گرفته و عبارت احتمالی درست را برمیگرداند

In [125]:
def ModifyText(Text):
    tokens = Text.split(' ')
    output = ""
    for t in tokens:
        ct = GetTrueWord(t)
        output += ct + " "
    return output

مثال

In [135]:
myText = "سهامم و بوورس"
print(ModifyText(myText))

سهام و بورس 


در نهایت تابع زیر با ترکیب تمامی روش ها یک پرس و جو انجام میدهد

In [136]:
def Query_2(Text):
    fixed_Text = ModifyText(Text)
    return Query(fixed_Text)

In [139]:
myText = "سهامم و بوورس"
Query_2(myText)

[13, 15, 16, 17, 18]